In [ ]:
!pip install -q gym numpy gymnasium stable-baselines3

In [ ]:
import carla
import os
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from time import sleep
import queue

In [ ]:
class CarlaEnv(gym.Env):

    def __init__(self, params):
        # Initial parameters
        self.desired_speed = params['desired_speed']
        self.number_of_vehicles = params['number_of_vehicles']
        self.number_of_walkers = params['number_of_walkers']

        # Destination
        self.dests = None

        # action space
        # load in the the discrete action space for each type of output
        self.discrete_act[params['discrete_acc'], params['discrete_steer'], params['discrete_brake'])
        self.n_acc = len(self.discrete_act[0])
        self.n_steer = len(self.discrete_act[1])
        self.n_brake = len(self.discrete_act[2])
        # Accel and steer are seperated as it makes no sense to use both at same time
        self.action_space = spaces.Discrete((self.n_acc*self.n_steer) + (self.n_steer*self.n_brake))

        # observation space
        observation_space_dict = {
            'camera': spaces.Box(
        }
        
        